In [19]:
import pandas as pd
import numpy as np
import math

In [20]:
art = pd.read_csv('data/shared_articles.csv')
art.head(1)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


In [21]:
articles_df = art[art.eventType == 'CONTENT SHARED']
articles_df.shape[0]

3047

In [22]:
interactions_df = pd.read_csv('data/users_interactions.csv')

In [23]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

C:\Users\Администратор.000\AppData\Local\Temp\ipykernel_2740\1305389506.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles_df.contentId = articles_df.contentId.astype(str)


In [24]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [25]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type[x])
interactions_df['eventStrength'].mean()

1.2362885828078327

In [26]:
users_interactions_count_df = (interactions_df.groupby(['personId', 'contentId']).first().reset_index().groupby('personId').size())
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print(len(users_with_enough_interactions_df))

1140


In [27]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId, users_with_enough_interactions_df)]
print(interactions_from_selected_users_df.shape)

(69868, 9)


In [30]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (interactions_from_selected_users_df.groupby(['personId', 'contentId']).eventStrength.sum().apply(smooth_user_preference).reset_index().set_index(['personId', 'contentId']))
interactions_full_df['last_timestamp'] = (interactions_from_selected_users_df.groupby(['personId', 'contentId'])['timestamp'].max())
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df['last_timestamp'].mean()

1470605340.0403006

In [31]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(len(interactions_train_df))

29325


In [32]:
final_df = (
    interactions_train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (interactions_test_df.reset_index().groupby('personId')['contentId'].agg(lambda x: list(x)))

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


In [33]:
popular = (
    interactions_train_df
    .groupby('contentId')
    .eventStrength.sum().reset_index()
    .sort_values('eventStrength', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

In [35]:
top_k = 10
 
final_df['popular'] = (final_df.true_train.apply(lambda x: popular[~np.in1d(popular, x)][:top_k]))

def calc_precision(column):
    return (final_df.apply(lambda row: len(set(row['true_test']).intersection(set(row[column]))) / min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

calc_precision('popular')

0.006454207722621089